In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline
import os
import ast
from helpers import *


In [4]:
data_folder = "C:/Users/cricl/PROJADA/"

In [5]:
movie_col_name = [
    "Movie_name",
    "Movie_release_date",
    "Movie_box_office_revenue",
    "Movie_runtime",
    "Movie_languages", 
    "Movie_countries", 
    "Movie_genres",
    "Actor_gender",
    "Actor_height_m",
    "Actor_ethnicity",
    "Actor_name",
    "Actor_age_at_movie_release",
    "Main_genre",
    "Main_country"    ]

In [6]:
def remove_outlier(df, col_name,threshold):
    for i in df[col_name].unique():
        values = df[df[col_name]== i][col_name].value_counts()
        if values.values< threshold:
            df = df[df[col_name] != i]
    return df


In [7]:
df = pd.read_csv(data_folder+'Big_Cleaned_dataset.csv', usecols=movie_col_name)
threshold = 100
col_name = "Main_country"
df = remove_outlier(df,col_name,threshold)
America = ['Canada','United States of America','Mexico','Argentina','Brazil']
Europa= ['United Kingdom','France','Germany','Poland','Finland','Hungary','Spain','Italy','Finland',
'Ireland','Denmark','Sweden','Norway','Netherlands','Czech Republic', 'Switzerland','Portugal','Belgium', 'Austria','England']
Asia = ['India','Japan','Hong Kong','South Korea','Pakistan','Turkey','Philippines','Russia', 'Thailand', 'Malaysia', 'China',
       'Indonesia']
Africa = ['South Africa']
Oceania = ['Australia']
Deadcountries = ['Soviet Union','Yugoslavia','West Germany','Weimar Republic','Czechoslovakia', 'German Democratic Republic']
df = df[~df['Main_country'].isin(Deadcountries)]



In [8]:
#categorical_columns = ['Main_genre','Main_country','Actor_gender']
continent = [America,Asia,Europa,Africa,Oceania]
continent_str =["America","Asia","Europa","Africa","Oceania"]
for i in range(0,len(continent)):
    df.loc[df.Main_country.isin(continent[i]),'continent'] = continent_str[i]
df.head()

,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Actor_gender,Actor_height_m,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Main_genre,Main_country,continent
0,Ghosts of Mars,2001.0,14010832.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",F,1.620,NaN,Wanda De Jesus,42.0,Thriller,United States of America,America
1,Ghosts of Mars,2001.0,14010832.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",F,1.780,/m/044038p,Natasha Henstridge,27.0,Thriller,United States of America,America
2,Ghosts of Mars,2001.0,14010832.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",M,1.727,/m/0x67,Ice Cube,32.0,Thriller,United States of America,America
3,Ghosts of Mars,2001.0,14010832.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",M,1.750,NaN,Jason Statham,33.0,Thriller,United States of America,America
4,Ghosts of Mars,2001.0,14010832.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",F,1.650,NaN,Clea DuVall,23.0,Thriller,United States of America,America


In [9]:


threshold = 4700 #4700 for big one
col_name = "Main_genre"
df = remove_outlier(df,col_name,threshold)
test = df.Main_genre
a=test.value_counts()
print(a)

Main_genre
Thriller            30558
Drama               19208
Crime Fiction       16761
Romantic comedy     13803
Short Film           7288
Romance Film         7106
Action/Adventure     5800
Science Fiction      5432
Comedy               5424
Parody               4824
Name: count, dtype: int64


In [10]:
categorical_columns = ['Main_genre','continent','Actor_gender']
df = pd.get_dummies(df, columns=categorical_columns, drop_first= True)

df = df.drop(columns=['Movie_name','Movie_languages','Actor_name','Main_country','Movie_countries','Movie_genres','Actor_ethnicity','Actor_height_m'])
df.columns

Index(['Movie_release_date', 'Movie_box_office_revenue', 'Movie_runtime',
       'Actor_age_at_movie_release', 'Main_genre_Comedy',
       'Main_genre_Crime Fiction', 'Main_genre_Drama', 'Main_genre_Parody',
       'Main_genre_Romance Film', 'Main_genre_Romantic comedy',
       'Main_genre_Science Fiction', 'Main_genre_Short Film',
       'Main_genre_Thriller', 'continent_America', 'continent_Asia',
       'continent_Europa', 'continent_Oceania', 'Actor_gender_M',
       'Actor_gender_Unknown'],
      dtype='object')

In [11]:
#1st analysis
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()d
# normalise only selected columns
#dfstand[cols] = sc.fit_transform(df[cols])
#df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.replace(' ', '')
dfstand=df
dfstand['Movie_release_date'] = (dfstand['Movie_release_date']- dfstand['Movie_release_date'].mean())/dfstand['Movie_release_date'].std()
dfstand['Movie_box_office_revenue'] = (dfstand['Movie_box_office_revenue']- dfstand['Movie_box_office_revenue'].mean())/dfstand['Movie_box_office_revenue'].std()
dfstand['Movie_runtime'] = (dfstand['Movie_runtime']- dfstand['Movie_runtime'].mean())/dfstand['Movie_runtime'].std()
dfstand['Actor_age_at_movie_release'] = (dfstand['Actor_age_at_movie_release']- dfstand['Actor_age_at_movie_release'].mean())/dfstand['Movie_release_date'].std()
dfstand= dfstand[df['Movie_box_office_revenue'].notna()]
dfstand= dfstand[df['Movie_runtime'].notna()]


C:\Users\cricl\AppData\Local\Temp\ipykernel_16372\3897648596.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfstand= dfstand[df['Movie_runtime'].notna()]


In [12]:
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
dfstand['Actor_gender_M']=dfstand['Actor_gender_M'].astype(int)
mod = smf.logit(formula='Actor_gender_M ~  Movie_release_date * Movie_box_office_revenue + Movie_runtime + \
                       Actor_age_at_movie_release+ C(Main_genre_Comedy)+\
                       C(Main_genre_CrimeFiction)+C(Main_genre_Drama)+ C(Main_genre_Parody)+\
                       C(Main_genre_RomanceFilm)+C(Main_genre_Romanticcomedy)+\
                       C(Main_genre_ScienceFiction)+ C(Main_genre_ShortFilm)+\
                       C(Main_genre_Thriller+continent_America)+ C(continent_Asia)+\
                       C(continent_Europa)+ C(continent_Oceania)', data=dfstand)
res = mod.fit()
dfstand['Propensity_score'] = res.predict()
print(res.summary())
dfstand.head()

Optimization terminated successfully.
         Current function value: 0.597300
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:         Actor_gender_M   No. Observations:                42322
Model:                          Logit   Df Residuals:                    42304
Method:                           MLE   Df Model:                           17
Date:                Thu, 12 Dec 2024   Pseudo R-squ.:                 0.05367
Time:                        10:46:42   Log-Likelihood:                -25279.
converged:                       True   LL-Null:                       -26712.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                         coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                   

,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Actor_age_at_movie_release,Main_genre_Comedy,Main_genre_CrimeFiction,Main_genre_Drama,Main_genre_Parody,Main_genre_RomanceFilm,Main_genre_Romanticcomedy,Main_genre_ScienceFiction,Main_genre_ShortFilm,Main_genre_Thriller,continent_America,continent_Asia,continent_Europa,continent_Oceania,Actor_gender_M,Actor_gender_Unknown,Propensity_score
0,0.655511,-0.440804,-0.093514,3.060704,False,False,False,False,False,False,False,False,True,True,False,False,False,0,False,0.724222
1,0.655511,-0.440804,-0.093514,-11.939296,False,False,False,False,False,False,False,False,True,True,False,False,False,0,False,0.606711
2,0.655511,-0.440804,-0.093514,-6.939296,False,False,False,False,False,False,False,False,True,True,False,False,False,1,False,0.648132
3,0.655511,-0.440804,-0.093514,-5.939296,False,False,False,False,False,False,False,False,True,True,False,False,False,1,False,0.656177
4,0.655511,-0.440804,-0.093514,-15.939296,False,False,False,False,False,False,False,False,True,True,False,False,False,0,False,0.572398


In [42]:
import networkx as nx
dftest = dfstand.sample(n=500)
dftest.head()
control = dftest[dftest.Actor_gender_M == 0]
treat =  dftest[dftest.Actor_gender_M == 1 ]
def computediff(prop1,prop2):
    return np.abs(prop1-prop2)
def compute_similarity(prop1,prop2):
    return 1-np.abs(prop1-prop2)

G = nx.Graph()
for treat_id,treat_row in treat.iterrows():
    for control_id, control_row in control.iterrows():
        diff= computediff(control_row["Propensity_score"],treat_row["Propensity_score"])
        if (diff<0.001):
            similarity =  compute_similarity(control_row["Propensity_score"],treat_row["Propensity_score"])
            G.add_weighted_edges_from([(control_id, treat_id, similarity)])
matching = nx.max_weight_matching(G)
#matched = [i[0] for i in list(matching)] + [i[1] for i in list(matching)]
print("Number of successful matching:",len(matching))
print(matching)

Number of successful matching: 102
{(58946, 149322), (49446, 58795), (196691, 110509), (97148, 199239), (70075, 70361), (54063, 78349), (44147, 96978), (60046, 154518), (57562, 125259), (31802, 42761), (22481, 198385), (176875, 179779), (181384, 114934), (94372, 93841), (182758, 184825), (84019, 30224), (23165, 157189), (138856, 97411), (2869, 68981), (114180, 57953), (37777, 82336), (89953, 134046), (457, 169201), (37864, 114252), (30895, 160791), (7415, 124938), (14162, 111454), (46713, 35867), (179754, 39001), (61243, 24515), (116984, 190762), (179784, 62505), (165269, 31090), (80769, 25641), (138762, 69883), (181443, 181418), (36222, 114411), (129466, 28707), (135239, 41386), (111560, 54949), (108565, 71051), (7416, 49425), (96332, 156329), (32049, 138320), (15483, 78532), (67125, 129909), (6422, 129914), (172268, 54067), (57036, 37462), (60293, 56356), (163303, 158071), (43651, 155955), (179086, 174222), (70518, 192366), (89788, 49771), (27055, 56733), (49819, 16947), (169648, 146

In [74]:
dftest.iloc[0]

Movie_release_date           -0.432269
Movie_box_office_revenue      -0.29049
Movie_runtime                 0.709781
Actor_age_at_movie_release   -5.939296
Main_genre_Comedy                False
Main_genre_CrimeFiction          False
Main_genre_Drama                 False
Main_genre_Parody                False
Main_genre_RomanceFilm           False
Main_genre_Romanticcomedy        False
Main_genre_ScienceFiction        False
Main_genre_ShortFilm             False
Main_genre_Thriller               True
continent_America                 True
continent_Asia                   False
continent_Europa                 False
continent_Oceania                False
Actor_gender_M                       0
Actor_gender_Unknown             False
Propensity_score              0.703517
Name: 89788, dtype: object

,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Actor_age_at_movie_release,Main_genre_Comedy,Main_genre_CrimeFiction,Main_genre_Drama,Main_genre_Parody,Main_genre_RomanceFilm,Main_genre_Romanticcomedy,Main_genre_ScienceFiction,Main_genre_ShortFilm,Main_genre_Thriller,continent_America,continent_Asia,continent_Europa,continent_Oceania,Actor_gender_M,Actor_gender_Unknown,Propensity_score
89788,-0.432269,-0.290490,0.709781,-5.939296,False,False,False,False,False,False,False,False,True,True,False,False,False,0,False,0.703517
165269,0.693020,-0.327316,-0.260867,2.060704,False,False,False,False,False,True,False,False,False,True,False,False,False,1,False,0.596083
133817,0.768039,0.705348,-0.126985,24.060704,False,False,False,False,False,False,False,False,True,True,False,False,False,0,False,0.850601
61913,0.730530,-0.171864,-0.662515,-2.939296,False,False,False,False,False,False,False,False,True,True,False,False,False,0,False,0.676057
116942,-0.469778,0.233837,0.475487,2.060704,False,False,False,False,False,False,False,False,True,False,False,True,False,1,False,0.752583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153497,0.317924,-0.531633,-0.361279,-12.939296,False,True,False,False,False,False,False,False,False,True,False,False,False,0,False,0.632743
34034,0.017847,-0.361969,0.877135,3.060704,False,True,False,False,False,False,False,False,False,True,False,False,False,1,False,0.770639
41981,-1.032423,-0.499086,0.508958,4.060704,False,False,True,False,False,False,False,False,False,True,False,False,False,1,False,0.692701
32802,0.655511,-0.497229,-0.193926,-13.939296,False,False,False,False,False,False,False,False,True,True,False,False,False,0,False,0.588228
